In [14]:
!pip install ultralytics

In [25]:
import ultralytics
ultralytics.__version__

'8.2.12'

In [26]:
import cv2
import time
import pandas as pd
from ultralytics import YOLO
from tracker import*

model=YOLO('yolov8s.pt')

In [27]:
tracker=Tracker()
count=0

In [28]:
class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 
              'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

In [35]:
cap=cv2.VideoCapture('ggg.mp4')
if not cap.isOpened():
    print("Error opening video file")
    exit(0)

In [36]:
fps = cap.get(cv2.CAP_PROP_FPS)
print("Frames per Second:", fps)

Frames per Second: 25.0


In [37]:
down = {}
up = {}
counter_down = []
counter_up = []

In [38]:
red_line_start = (650, 100)
red_line_end = (950, 175)
blue_line_start = (389, 178)
blue_line_end = (885, 342)
distance = 5  # meters
offset = 5
frame_count = 0

In [39]:
def is_crossing_line(line_start, line_end, cx, cy):
    """Check if a point (cx, cy) crosses a line defined by (line_start, line_end)"""
    x1, y1 = line_start
    x2, y2 = line_end
    if x2 == x1:  # Vertical line
        return abs(cx - x1) < offset
    m = (y2 - y1) / (x2 - x1)
    b = y1 - m * x1
    line_y = m * cx + b
    crossing = abs(cy - line_y) < offset
    print(f"Checking line crossing: cx={cx}, cy={cy}, line_y={line_y}, crossing={crossing}")
    return crossing


In [40]:
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1
    frame = cv2.resize(frame, (1020, 500))

    
    results = model.predict(frame)
    a = results[0].boxes.data
    a = a.detach().cpu().numpy()
    px = pd.DataFrame(a).astype("float")

    list = []

    for index, row in px.iterrows():
        x1 = int(row[0])
        y1 = int(row[1])
        x2 = int(row[2])
        y2 = int(row[3])
        d = int(row[5])
        c = class_list[d]
        if 'car' in c:
            list.append([x1, y1, x2, y2])

    bbox_id = tracker.update(list)

    for bbox in bbox_id:
        x3, y3, x4, y4, id = bbox
        cx = int((x3 + x4) / 2)
        cy = int((y3 + y4) / 2)

        cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
        cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 0, 255), 2)

        # Track vehicle movement down
        if is_crossing_line(red_line_start, red_line_end, cx, cy):
            down[id] = frame_count
            print(f"Car {id} crossed the red line at frame {frame_count}")

        if id in down:
            if is_crossing_line(blue_line_start, blue_line_end, cx, cy):
                frames_elapsed = frame_count - down[id]
                elapsed_time = frames_elapsed / fps
                print(f"Down: frames_elapsed={frames_elapsed}, elapsed_time={elapsed_time}")
                if elapsed_time > 0:  # To avoid division by zero
                    a_speed_ms = distance / elapsed_time
                    a_speed_kh = a_speed_ms * 3.6
                    cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)
                    cv2.putText(frame, str(id), (x3, y3), cv2.FONT_HERSHEY_COMPLEX, 0.6, (255, 255, 255), 1)
                    cv2.putText(frame, str(int(a_speed_kh)) + ' Km/h', (x4, y4), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
                    print(f"Speed (down) for car {id}: {a_speed_kh} Km/h")  # Additional debug statement

        # Track vehicle movement up
        if is_crossing_line(blue_line_start, blue_line_end, cx, cy):
            up[id] = frame_count
            print(f"Car {id} crossed the blue line at frame {frame_count}")

        if id in up:
            if is_crossing_line(red_line_start, red_line_end, cx, cy):
                frames_elapsed = frame_count - up[id]
                elapsed_time = frames_elapsed / fps
                print(f"Up: frames_elapsed={frames_elapsed}, elapsed_time={elapsed_time}")
                if elapsed_time > 0:  # To avoid division by zero
                    a_speed_ms = distance / elapsed_time
                    a_speed_kh = a_speed_ms * 3.6

                    if id not in counter_up:
                        counter_up.append(id)
                        print(f"Car {id} crossed the red line at frame {frame_count}")
                        print(f"Frames elapsed (up): {frames_elapsed}, Elapsed time: {elapsed_time}")
                        print(f"Car {id} speed (up): {int(a_speed_kh)} Km/h")
                    cv2.rectangle(frame, (x3, y3), (x4, y4), (0, 255, 0), 2)
                    cv2.putText(frame, str(id), (x3, y3), cv2.FONT_HERSHEY_COMPLEX, 0.6, (255, 255, 255), 1)
                    cv2.putText(frame, str(int(a_speed_kh)) + ' Km/h', (x4, y4), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)
                    print(f"Speed (up) for car {id}: {a_speed_kh} Km/h")  # Additional debug statement

    text_color = (255, 255, 255)  # white color for text
    red_color = (0, 0, 255)  # Red color for lines
    blue_color = (255, 0, 0)  # Blue color for lines
        
    cv2.line(frame, red_line_start, red_line_end, red_color, 3)
    cv2.putText(frame, 'Red Line', red_line_start, cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)
    cv2.line(frame, blue_line_start, blue_line_end, blue_color, 3)
    cv2.putText(frame, 'Blue Line', blue_line_start, cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    cv2.putText(frame, 'Going Down - ' + str(len(counter_down)), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)
    cv2.putText(frame, 'Going Up - ' + str(len(counter_up)), (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA)

    cv2.imshow("frames", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


0: 320x640 1 person, 2 cars, 206.4ms
Speed: 2.0ms preprocess, 206.4ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)
Checking line crossing: cx=134, cy=56, line_y=-29.0, crossing=False
Checking line crossing: cx=134, cy=56, line_y=93.68548387096774, crossing=False
Checking line crossing: cx=919, cy=50, line_y=167.25, crossing=False
Checking line crossing: cx=919, cy=50, line_y=353.241935483871, crossing=False

0: 320x640 1 person, 2 cars, 335.1ms
Speed: 12.0ms preprocess, 335.1ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)
Checking line crossing: cx=134, cy=56, line_y=-29.0, crossing=False
Checking line crossing: cx=134, cy=56, line_y=93.68548387096774, crossing=False
Checking line crossing: cx=919, cy=50, line_y=167.25, crossing=False
Checking line crossing: cx=919, cy=50, line_y=353.241935483871, crossing=False

0: 320x640 1 person, 2 cars, 147.6ms
Speed: 2.0ms preprocess, 147.6ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)